# Selenium과 BeautifulSoup을 이용한 KFC 매장 정보 수집

### 라이브러리

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib.request as req
from urllib import parse
import pandas as pd
import time

### Web Driver

In [10]:
chrome_options=webdriver.ChromeOptions()
driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.kfckorea.com/store/findStore')

___

## 매장 정보 수집

In [11]:
#지역검색: //*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/ul/li[2]/a
#시도: //*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[1]
#구군: //*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[2]

region_search=driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/ul/li[2]/a').click()
sido_list_raw=driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]')
sido_list=sido_list_raw.find_elements(By.TAG_NAME,'option')
sido_names=[i.text for i in sido_list]
element=driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[1]')
time.sleep(2)
element.send_keys(sido_names[0])

address_total=[]

for l in sido_names:
    element.send_keys(l)
    gugun_names=[]
    gugun_list_raw=driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[2]')
    gugun_list=gugun_list_raw.find_elements(By.TAG_NAME,'option')
    gugun_names=[a.text for a in gugun_list]
    gugun_names.remove('구,군')
    print(gugun_names)
    element2=driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[2]')
    for i in gugun_names:
        time.sleep(1)
        element2.send_keys(i)
        html=driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        gu_address=soup.select("#app > div:nth-child(2) > div > div > div.open > div > div.detail > div > div > div > div > div:nth-child(3) > div.swiper-wrapper > div > ul > li:nth-child(3)")
        gu_address_text=[m.text for m in gu_address]
        shop_names=soup.select("#app > div:nth-child(2) > div > div > div.open > div > div.detail > div > div > div > div > div:nth-child(3) > div.swiper-wrapper > div > ul > li.top > a")
        shop_names_text=[n.text for n in shop_names]
        for j,k in zip(gu_address_text,shop_names_text):
            address_total.append((j.split()[0],j.split()[1],j,k))
            print((j.split()[0],j.split()[1],j,k))
        time.sleep(1)

[]
['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']
('서울', '강남구', '서울 강남구 학동로 219 (논현동) ', '학동역2')
('서울', '강남구', '서울 강남구 도곡로 517 (대치동) 서연빌딩 1층', '대치동')
('서울', '강남구', '서울 강남구 영동대로 513 (삼성동) B1층 H102호', '코엑스MALL')
('서울', '강남구', '서울 강남구 일원로 28 (일원동) ', '일원동')
('서울', '강남구', '서울 강남구 논현로 509 (역삼동) 지상1층2층', '역삼역')
('서울', '강남구', '서울 강남구 학동로 402 (삼성동) 천마빌딩 103호 ', '강남구청역 (구: 삼성동)')
('서울', '강동구', '서울 강동구 양재대로 1651 (명일동) ', '명일DI점')
('서울', '강동구', '서울 강동구 천호옛길 88 (성내동) G-lite빌딩 1층', '천호역')
('서울', '강동구', '서울 강동구 양재대로 1390 (둔촌동) ', '둔촌동')
('서울', '강북구', '서울 강북구 도봉로10길 12 (미아동) ', '미아사거리역')
('서울', '강서구', '서울 강서구 공항대로 271 (마곡동) 마곡동 723-2', '발산역')
('서울', '강서구', '서울 강서구 강서로 162 (화곡동) ', '화곡역')
('서울', '강서구', '서울 강서구 양천로 583 (염창동) 우림블루나인비즈니스센터1층', '염창동')
('서울', '강서구', '서울 강서구 등촌로 19 (화곡동) ', '목동사거리')
('서울', '관악구', '서울 관악구 신림로 318 (신림동, 청암두산위브센티움) ', '신림역')
('서울', '관악구', '서울 관악구 남

### CSV 파일로 변환

In [12]:
df=pd.DataFrame(data=address_total,columns=['광역시도','시군구','주소','지점명'])
df.to_csv("../Data/KFC.csv",encoding='utf-8',index=False)

___
## KFC 매장의 위도와 경도 수집

In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import json

kfc=pd.read_csv('../Data/KFC.csv')
addressesx=[]
addressesy=[]

client_secret="LfPn2DBkJs8motLtQOouNwsF6OYi7rlfzQFUWN2F"
client_id="dhnl44bahk"

for address in list(kfc['주소']):
    encText = urllib.parse.quote(address)
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" + encText
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        entext = response_body.decode('utf-8')
        data1 = json.loads(entext)
        try:
            addressesx.append(data1['addresses'][0]['x'])
            addressesy.append(data1['addresses'][0]['y'])
        except:
            addressesx.append(0)
            addressesy.append(0)
    else:
        print("Error Code:" + rescode)

print((addressesx,addressesy))

(['127.0331486', '127.0629430', '127.0594275', '127.0803513', '127.0361160', '127.0418047', '127.1438128', '127.1258722', '127.1375999', '127.0308085', '126.8378241', '126.8410422', '126.8642592', '126.8634874', '126.9302485', '126.9615689', '126.9511469', '126.9430660', '127.0735712', '127.0949441', '126.8966116', '126.9027245', '126.8815811', '127.0639254', '127.0604045', '127.0762650', '127.0613340', '127.0129919', '127.0606824', '126.9249684', '126.9599920', '126.9483698', '126.8898190', '126.9234581', '126.9223494', '126.9352082', '126.9822747', '126.9827042', '127.0260957', '127.0352051', '127.0283705', '127.0150234', '127.0139865', '127.0048933', '127.0384812', '127.0291686', '127.0176612', '127.0981638', '127.0736261', '127.0846691', '127.1068556', '127.1351390', '126.8522656', '126.9016531', '126.9076660', '126.9204076', '126.9298331', '126.9716850', '127.0049784', '126.9647415', '126.9219821', '126.9205529', '126.9286789', '126.9813949', '127.0107296', '127.0023149', '126.975

### CSV 파일로 저장

In [14]:
kfc['경도']=addressesx
kfc['위도']=addressesy
kfc.to_csv("../Data/KFC.csv",encoding='utf-8',index=False)

### 지도에 마커로 표시

In [15]:
kfc=pd.read_csv('../Data/KFC.csv')

In [16]:
import folium

kfc_map2=folium.Map(
    location=[37.55,126.98],
    zoom_start=8,
    tiles='Stamen Terrain'
    )

for lat,lng,name in zip(kfc.위도,kfc.경도,kfc.지점명):
    folium.Marker([lat,lng],popup=name).add_to(kfc_map2)

kfc_map2